In [ ]:
## NOTE: SOMETHING IS SCREWED UP WITH YOUR LOGIT PROBABILITIES...FIX THIS!!!

In [1]:
# import pandas as pd
# import numpy as np

# df_pred = pd.read_csv('../my_data/elo_pbp_with_surface_10_15.csv')
# df_pred[['elo_diff_538','sf_elo_diff_538','elo_prob_538','logit_elo_538_prob']]

In [15]:
import sys
sys.path.insert(0,'/Users/jacobgollub/Desktop/college/research/pbp_explorations/scripts/sackmann')
import tennisGameProbability,tennisMatchProbability,tennisSetProbability,tennisTiebreakProbability
from tennisMatchProbability import matchProb

def match_prob(sv0,sv1,server,s0,s1,g0,g1,p0,p1):
    if server:
        return matchProb(sv0,1-sv1,mv=s0,mw=s1,sv=g0,sw=g1,gv=p0,gw=p1)
    else:
        return 1 - matchProb(sv1,1-sv0,mv=s1,mw=s0,sv=g1,sw=g0,gv=p1,gw=p0)

In [3]:
import pandas as pd
import numpy as np

df_pred = pd.read_csv('../my_data/feature_df_pbp3_10_2.csv')
del df_pred['Unnamed: 0']

In [4]:
import findspark
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc = SparkContext()
sqlContext = SQLContext(sc)
# findspark.init()

In [8]:
#sc.stop()

In [7]:
train = sqlContext.createDataFrame(df_pred)

KeyboardInterrupt: 

In [ ]:
train.show()

In [1]:
# TO DO: analyze different methods, especially at different points of the match

# TO DO: test a grid search of averaging historical with overall player percentages and see if any split (50-50,70-30,etc)
# then, run grid search on alpha parameter, for beta distribution
# run grid search on coefficients for combining elo/s_elo with serve percentages

# then, apply all these methods to find the probability of winning the next point, game, set

In [1]:
import pandas as pd
import numpy as np
# f_av = 0.646775
# g_av = 1 - f_av

import sys
sys.path.insert(0,'/Users/jacobgollub/Desktop/college/research/pbp_explorations/scripts/sackmann')
import tennisGameProbability,tennisMatchProbability,tennisSetProbability,tennisTiebreakProbability
from tennisMatchProbability import matchProb

df_pred = pd.read_csv('../my_data/feature_df_pbp3_10_15.csv')
del df_pred['Unnamed: 0']
# can run beta experiments based off of p(s)=.64 as well as klaassen and JS serve percentages
alphas = [1,10,100,200,300,400,500,600,1000]
col = 's_kls_logit_elo_538'
for alpha in alphas:
    df_pred['p0_'+col+'_a_'+str(alpha)] = (df_pred['p0_'+col]*alpha+df_pred['p0_swp'])/(alpha+df_pred['p0_sp'])
    df_pred['p1_'+col+'_a_'+str(alpha)] = (df_pred['p1_'+col]*alpha+df_pred['p1_swp'])/(alpha+df_pred['p1_sp'])

In [ ]:
for i,row in df_pred.iterrows():
    if row['server']:
        probs[3][i] = matchProb(row['p0_'+col],1-row['p1_'+col],mv=int(row['sets_0']),mw=int(row['sets_1']), \
                            sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
    else:
        probs[3][i] = 1 - matchProb(row['p1_'+col],1-row['p0_'+col],mv=int(row['sets_1']),mw=int(row['sets_0']), \
                            sv=int(row['games_1']),sw=int(row['games_0']),gv=int(row['points_1']),gw=int(row['points_0']))
df_pred['klaassen_prob_logit_elo'] = probs[3]
df_pred.to_csv('../my_data/feature_df_pbp3_10_15_probs.csv')

In [2]:
import time
start = time.clock()

probs = np.zeros([15,len(df_pred)])
for i,row in df_pred.iterrows():
    if row['server']:
#         probs[0][i] = matchProb(f_av,g_av,mv=int(row['sets_0']),mw=int(row['sets_1']),sv=int(row['games_0']), \
#                                 sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
#         probs[1][i] = matchProb(row['p0_s_kls'],1-row['p1_s_kls'],mv=int(row['sets_0']),mw=int(row['sets_1']), \
#                                 sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
        probs[2][i] = matchProb(row['p0_s_kls_JS'],1-row['p1_s_kls_JS'],mv=int(row['sets_0']),mw=int(row['sets_1']), \
                                sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
#         probs[3][i] = matchProb(row['p0_s_kls_adj'],1-row['p1_s_kls_adj'],mv=int(row['sets_0']),mw=int(row['sets_1']), \
#                                 sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
        
        probs[4][i] = matchProb(row['p0_s_kls_adj_JS'],1-row['p1_s_kls_adj_JS'],mv=int(row['sets_0']),mw=int(row['sets_1']), \
                                sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
        probs[5][i] = matchProb(row['p0_s_kls_elo'],1-row['p1_s_kls_elo'],mv=int(row['sets_0']),mw=int(row['sets_1']), \
                                sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
        for k,alpha in enumerate(alphas):
            probs[k+6][i] = matchProb(row['p0_'+col+'_a_'+str(alpha)],1-row['p1_'+col+'_a_'+str(alpha)],mv=int(row['sets_0']),mw=int(row['sets_1']), \
                            sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
    else:
#         probs[0][i] = 1-matchProb(f_av,g_av,mv=int(row['sets_1']),mw=int(row['sets_0']),sv=int(row['games_1']), \
#                                 sw=int(row['games_0']),gv=int(row['points_1']),gw=int(row['points_0']))
#         probs[1][i] = 1-matchProb(row['p1_s_kls'],1-row['p0_s_kls'],mv=int(row['sets_1']),mw=int(row['sets_0']), \
#                                 sv=int(row['games_1']),sw=int(row['games_0']),gv=int(row['points_1']),gw=int(row['points_0']))
        probs[2][i] = 1-matchProb(row['p1_s_kls_JS'],1-row['p0_s_kls_JS'],mv=int(row['sets_1']),mw=int(row['sets_0']), \
                                sv=int(row['games_1']),sw=int(row['games_0']),gv=int(row['points_1']),gw=int(row['points_0']))
#         probs[3][i] = 1-matchProb(row['p1_s_kls_adj'],1-row['p0_s_kls_adj'],mv=int(row['sets_1']),mw=int(row['sets_0']), \
#                                 sv=int(row['games_1']),sw=int(row['games_0']),gv=int(row['points_1']),gw=int(row['points_0']))
        probs[4][i] = 1-matchProb(row['p1_s_kls_adj_JS'],1-row['p0_s_kls_adj_JS'],mv=int(row['sets_1']),mw=int(row['sets_0']), \
                                sv=int(row['games_1']),sw=int(row['games_0']),gv=int(row['points_1']),gw=int(row['points_0']))
        probs[5][i] = 1-matchProb(row['p1_s_kls_elo'],1-row['p0_s_kls_elo'],mv=int(row['sets_1']),mw=int(row['sets_0']), \
                                sv=int(row['games_1']),sw=int(row['games_0']),gv=int(row['points_1']),gw=int(row['points_0']))
        for k,alpha in enumerate(alphas):
            probs[k+6][i] = 1-matchProb(row['p1_'+col+'_a_'+str(alpha)],1-row['p0_'+col+'_a_'+str(alpha)],mv=int(row['sets_1']),mw=int(row['sets_0']), \
                            sv=int(row['games_1']),sw=int(row['games_0']),gv=int(row['points_1']),gw=int(row['points_0']))
    
    if i%100000==0:
        print i

# df_pred['sackmann_prob'],df_pred['klaassen_prob'],df_pred['klaassen_prob_JS'] = probs[0],probs[1],probs[2]
# df_pred['klaassen_prob_adj'], df_pred['klaassen_prob_adj_JS'] = probs[3],probs[4]
df_pred['klaassen_prob_JS'], df_pred['klaassen_prob_adj_JS'] = probs[2],probs[4]
df_pred['klaassen_prob_elo'] = probs[5]
df_pred['klaassen_prob_a_1'],df_pred['klaassen_prob_a_10'] = probs[6],probs[7]
df_pred['klaassen_prob_a_100'], df_pred['klaassen_prob_a_200'] = probs[8],probs[9]
df_pred['klaassen_prob_a_300'], df_pred['klaassen_prob_a_400'] = probs[10],probs[11]
df_pred['klaassen_prob_a_500'], df_pred['klaassen_prob_a_600'] = probs[12],probs[13]
df_pred['klaassen_prob_a_1000'] = probs[14]
df_pred.to_csv('../my_data/feature_df_pbp3_10_15_probs.csv')
print time.clock()-start

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
24652.955841
